In [35]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain_core.prompts import (
    FewShotPromptTemplate
)
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
def get_embedding():
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
        )
    return embeddings

In [ ]:
#Fewshot을 위한 csv 파일을 불러옵니다.
train_df = pd.read_csv('train.csv')
train_df.drop('SAMPLE_ID', axis=1, inplace=True)
trainset = train_df.to_dict(orient='records')
print(trainset[:3])

example_prompt = PromptTemplate.from_template("Question: {Question}\nAnswer: {Answer}\nSource: {Source}")
print(example_prompt.invoke(trainset[1]).to_string())

to_vectorize = ["\n\n".join(example.values()) for example in trainset]

fewshow_vectors = FAISS.from_texts(to_vectorize, get_embedding(), metadatas=trainset)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=fewshow_vectors,
    k=3,
)
example_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)


In [33]:
test_df = pd.read_csv('test.csv')
test_df.drop('SAMPLE_ID', axis=1, inplace=True)
testset = test_df.to_dict(orient='records')
testset[1]

{'Source': '중소벤처기업부_혁신창업사업화자금(융자)',
 'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf',
 'Question': '중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?'}

In [32]:
ex_qa = example_prompt.invoke({"input": testset[1]['Question']}).to_string()
ex_qa

"Question: 창업사업화지원의 사업목적은 무엇인가?\nAnswer: 창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.\nSource: 중소벤처기업부_창업사업화지원\n\nQuestion: 언제 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 '기술혁신형 창업기업 지원사업'이 신설되었나요?\nAnswer: 기술혁신형 창업기업 지원사업(현, 예비창업패키지)'은 '18년에 신설되었습니다.\nSource: 중소벤처기업부_창업사업화지원\n\nQuestion: 2024년 중소벤처기업부의 창업사업화지원 사업에서 어떤 기관들이 사업시행주체로 참여하는가?\nAnswer: 중소벤처기업부, 창업진흥원이 사업시행주체이다.\nSource: 중소벤처기업부_창업사업화지원\n\nQuestion: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?"

In [56]:
pdf_directory = './train_source'
# PDF 파일들을 로드하여 분할한 뒤 텍스트를 벡터로 변환하여 DB에 저장
def load_chunks_make_docdb(pdf_directory):
    print("Loading PDF files from:", pdf_directory)
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf').replace('\\', '/'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)
    

    # 분할된 텍스트를 벡터로 변환하여 ChromaDB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    print("Done.", len(chunks), "chunks")
    
    print("Creating FAISS DB")
    """FAISS DB 생성"""
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=get_embedding())
    print("Done.")
    
    return db

# Load and index documents from both PDF and text files
train_vectordb =load_chunks_make_docdb(pdf_directory)



Loading PDF files from: ./train_source
Done. 1451 chunks
Creating FAISS DB


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Done.


In [63]:
print(trainset[1])
train_retriever = train_vectordb.as_retriever(search_type = "mmr",search_kwargs={'k': 3, 'fetch_k': 50})
retrieved_docs = train_retriever.invoke(testset[1]['Question'])

print(len(retrieved_docs))
retrieved_docs[0].page_content

{'Source': '1-1 2024 주요 재정통계 1권', 'Source_path': './train_source/1-1 2024 주요 재정통계 1권.pdf', 'Question': '2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?', 'Answer': '2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.'}
3


'정책지원성과향상(기금)중소벤처기업\n창업및진흥기\n금\n1200-1253 9,578\n성장안정자금(기금)중소벤처기업\n창업및진흥기\n금\n1200-1261 2,017,644\n신용보증기관출연 일반회계\n1200-1231 100,000\n    프로그램목표 Ⅱ-2. 중소기업의 글로벌 진출과 해외 판로 확보를 통해 성장을 \n도모한다.\n259,143\n중소기업수출\n촉진지원\n중소기업수출역량강화 일반회계\n3100-3131 111,860\n중소기업해외시장개척지원 일반회계\n3100-3133 101,726\n해외산업협력지원(기금) 중소벤처기업\n3100-3151 1,847'

In [ ]:
train_chain = (
    {}
)

In [ ]:
full_template = """{introduction}

{example}

{context}

{start}
"""

full_prompt = PromptTemplate.from_template(full_template)

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("What are autonomous agents?")

In [ ]:

prompt ="""
    This is an example of a Financial Question and Answering system.
    The system is designed to answer questions related to context.
    The context is a list of financial documents.
    The question is a financial question.
    The answer is the answer to the question.
    
"""
prompt_template = PromptTemplate.from_template("Question: {Question}\nAnswer: {Answer}\nSource: {Source}")

ex_chain = (
    {
        "context": retriever | format_docs,
        "input": testset[1]['Question'],
    }
    | example_prompt
    | llm
    | StrOutputParser()
)

In [68]:
for i in tqdm(range(len(testset))):
    ex_qa = example_prompt.invoke({"input": testset[i]['Question']}).to_string()

    
    break

  0%|          | 0/98 [00:00<?, ?it/s]

[{'Question': '창업사업화지원 사업의 2022년 결산 기준 예산 규모는 얼마인가?', 'Answer': '2022년 창업사업화지원 사업의 결산 기준 예산 규모는 432,824백만원이다.', 'Source': '중소벤처기업부_창업사업화지원'}, {'Question': '2023년 예산안을 기준으로 재정융자 규모는 얼마이며, 총지출 대비 어느 정도를 차지하나요?', 'Answer': '2023년도 예산안 기준 17개의 부처가 50개 사업을 통해 총 41.8조원 규모의 융자사업을 수행하고 있으며, 총지출 대비 6.6%를 차지합니다.', 'Source': '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》'}, {'Question': '2023년 예산안 기준 가장 큰 융자지출 규모는 어느 분야이며, 비중은 얼마인가?', 'Answer': '사회복지 분야로 26조 8,963억원이며, 비중은 64.3%이다.', 'Source': '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》', 'context': '며,총지출대비 6.6%\n- 재정융자사업은 특별회계와 기금으로 구성되는데 기금이 절대적인 비중을 차지. 2023년 전체 재정융자사업 \n재원가운데 기금이 98.2%(41조 871억원), 특별회계는 1.8%(7,509억원)\n- 전년대비(2022년 본예산 대비) 전체 재정융자 규모는 조 7,814억원 감소하였고 이 중 기금이 조 6,117억\n원,특별회계가 1,696억원 감소\n-  최근5년간 전년 대비 추이를 보면 2019~2021년은 융자 규모가 증가했으나, 2022년 2.2% 감소하였고 \n2023년은10.3%로 큰 폭의 감소를 보임03 \n주: 2019년부터 2021년까지는 결산 기준, 2022년은 본예산, 2023년은 예산안 기준. 융자 규모 추이 총지출 대비 융자지출 비중\n(단위: 조원,%) (단위: %)(분야별)취약계층 지원, 중소기업 지원 등에 대한 재정융자사업의 비중이 높음\n- 융자지출 규모가 가장 큰 분야는 사회복지 분야로 2023년